In [59]:
import sys
sys.path.append("../")
import data.directrepository as dr
import data.flightrepository as fr
import services.core.flightservice as fs
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
import importlib as im
import queue
import json
from enum import Enum
sys.path.append("../../")
import services.core.recoservice_df as reco
im.reload(reco)
from model.flight import Flight
from model.direct import Direct
import datetime
import os

im.reload(reco)
im.reload(fs)
im.reload(fr)



<module 'data.flightrepository' from 'C:\\Users\\Dave\\PycharmProjects\\FlightFinder\\data\\flightrepository.py'>

In [51]:
%%time
os.chdir("C:\\Users\\Dave\\PycharmProjects\\FlightFinder\\")

print("hello")

hello
Wall time: 1.01 ms


In [55]:

table = fr.FlightsRepository().get_flights_for_date("20190705")
table[:1]




,Airline,ArrivalTimeUTC,DepartTimeUTC,Duration,FlightNum,Price,FlyFrom,FlyTo,DataDate
0,O2,2019-10-11 13:51:00+00:00,2019-10-11 12:30:00+00:00,1h 21m,8069,2286,AGC,BWI,20190705


In [58]:
table.memory_usage()

Index                   80
Airline           14020298
ArrivalTimeUTC    55985608
DepartTimeUTC     55985608
Duration          14043970
FlightNum         13996402
Price             27992804
FlyFrom           14009226
FlyTo             14094186
DataDate           6998297
dtype: int64

In [57]:
fr.FlightsRepository().downcast_numerics(table)
#stable = fr.FlightsRepository().categorize_object_columns(table)
#stable[:1]
#stable.memory_usage()

In [61]:
fr.FlightsRepository().insert_flights(table)

In [29]:

#stable.dtypes


Index                   80
Airline           14020298
ArrivalTimeUTC    55985608
DepartTimeUTC     55985608
Duration          14043970
FlightNum         13996402
Price             27992804
FlyFrom           14009226
FlyTo             14094186
DataDate           6998297
dtype: int64

In [20]:
%%time
start = datetime.datetime(2019,10,15)
end = datetime.datetime(2019,10,15)
# flights = fr.FlightsRepository().get_latest_flights(start, end)
flights = fr.FlightsRepository().get_flights_for_dates(start, end)
flights[:1]

Wall time: 1min 51s


In [26]:
%%time
start = datetime.datetime(2019,10,15)
end = datetime.datetime(2019,10,15)
# flights = fr.FlightsRepository().get_latest_flights(start, end)
flights = fr.FlightsRepository().get_flights_for_dates_and_list(start, end, end)
flights[:1]

Wall time: 57.2 s


In [28]:
%%time
start = datetime.datetime(2019,10,15)
end = datetime.datetime(2019,10,15)
# flights = fr.FlightsRepository().get_latest_flights(start, end)
flights = fr.FlightsRepository().get_flights_for_dates_and_list(start, end, end)
flights[:1]

Wall time: 6.43 s


In [2]:
%%time
start = datetime.datetime(2019,10,15)
end = datetime.datetime(2019,10,15)
# flights = fr.FlightsRepository().get_latest_flights(start, end)
flights = fr.FlightsRepository().get_flights_for_dates_and_list(start, end, end)
flights[:1]

AttributeError: 'FlightsRepository' object has no attribute 'get_flights_for_dates_and_list'

In [49]:
flights.shape

(665, 9)

In [28]:
def datetime_handler(x):
    if isinstance(x, datetime.datetime):
        return x.isoformat()
    return x


start = datetime.datetime(2019,6,1)
end = datetime.datetime(2019,6,15)
flights = fs.FlightService().get_flights(start,end)
# flights["ArrivalTimeUTC"] = flights["ArrivalTimeUTC"]
# flights["DepartTimeUTC"] = flights["DepartTimeUTC"]

paths = reco.RecommendationService().get_recommendations(flights,["JFK","EWR","LGA"],["CTG"],[],2,10)
paths

list_of_paths = []
for path in paths:
    list_of_flights = []
    for flights in path:
        list_of_flights.append(flights.to_dict())
    list_of_paths.append(list_of_flights)
list_of_paths = json.dumps(list_of_paths, default=datetime_handler)
list_of_paths

    
    

Passed non-file path: storage\flights.parquet


KeyError: 'FlyTo'

In [29]:
flights = 

result = reco.RecommendationService().get_recommendations(flights, sources, targets, exclusions,2,10)
result.to_json(orient='records')

SyntaxError: invalid syntax (<ipython-input-29-0162c3217f7c>, line 1)

In [41]:
f1 = Flight("JFK", "LHR", 1000, "DU", "", datetime.datetime(2019, 5, 10, 11, 0, 0),
            datetime.datetime(2019, 5, 10, 6, 0, 0), "123", datetime.date.today())
f2 = Flight("JFK", "MIA", 50, "DU", "", datetime.datetime(2019, 5, 5, 11, 0, 0),
            datetime.datetime(2019, 5, 5, 6, 0, 0), "456", datetime.date.today())
f3 = Flight("MIA", "LHR", 100, "DU", "", datetime.datetime(2019, 5, 8, 11, 0, 0),
            datetime.datetime(2019, 5, 8, 6, 0, 0), "456", datetime.date.today())
f4 = Flight("LHR", "EWR", 450, "DU", "", datetime.datetime(2019, 5, 18, 11, 0, 0),
            datetime.datetime(2019, 5, 18, 6, 0, 0), "123", datetime.date.today())
f5 = Flight("LHR", "ATL", 200, "DU", "", datetime.datetime(2019, 5, 18, 11, 0, 0),
            datetime.datetime(2019, 5, 18, 6, 0, 0), "123", datetime.date.today())
f6 = Flight("ATL", "JFK", 300, "DU", "", datetime.datetime(2019, 5, 20, 11, 0, 0),
            datetime.datetime(2019, 5, 20, 6, 0, 0), "123", datetime.date.today())
f7 = Flight("EWR", "LHR", 1200, "DU", "", datetime.datetime(2019, 5, 16, 11, 0, 0),
            datetime.datetime(2019, 5, 16, 6, 0, 0), "123", datetime.date.today())
f8 = Flight("JFK", "LHR", 1200, "DU", "", datetime.datetime(2019, 5, 16, 11, 0, 0), #dup of f1 but higher price
            datetime.datetime(2019, 5, 16, 6, 0, 0), "123", datetime.date.today())
f9 = Flight("JFK", "LHR", 5, "DU", "", datetime.datetime(2019, 5, 10, 11, 0, 0), #dup of f1 but on same day
            datetime.datetime(2019, 5, 10, 6, 0, 0), "123", datetime.date.today())
ffs = [f1, f2, f3, f4, f5, f6, f7, f8,f9]
flights = pd.DataFrame.from_records([f.to_dict() for f in ffs])
df = pd.DataFrame()

sources = ["JFK","EWR"]
targets = ["LHR"]
exclusions = []

df = pd.DataFrame()
flights


,Airline,ArrivalTimeUTC,DataDate,DepartTimeUTC,Duration,FlightNum,FlyFrom,FlyTo,Price
0,DU,2019-05-10 11:00:00,2019-07-04,2019-05-10 06:00:00,,123,JFK,LHR,1000
1,DU,2019-05-05 11:00:00,2019-07-04,2019-05-05 06:00:00,,456,JFK,MIA,50
2,DU,2019-05-08 11:00:00,2019-07-04,2019-05-08 06:00:00,,456,MIA,LHR,100
3,DU,2019-05-18 11:00:00,2019-07-04,2019-05-18 06:00:00,,123,LHR,EWR,450
4,DU,2019-05-18 11:00:00,2019-07-04,2019-05-18 06:00:00,,123,LHR,ATL,200
5,DU,2019-05-20 11:00:00,2019-07-04,2019-05-20 06:00:00,,123,ATL,JFK,300
6,DU,2019-05-16 11:00:00,2019-07-04,2019-05-16 06:00:00,,123,EWR,LHR,1200
7,DU,2019-05-16 11:00:00,2019-07-04,2019-05-16 06:00:00,,123,JFK,LHR,1200
8,DU,2019-05-10 11:00:00,2019-07-04,2019-05-10 06:00:00,,123,JFK,LHR,5


In [6]:
start = datetime.datetime(2019,5,1)
end = datetime.datetime(2019,5,15)
flights_2 = flights[(flights['DepartTimeUTC'] > start) & (flights['DepartTimeUTC'] < end)  ]
flights_2

NameError: name 'flights' is not defined

In [22]:
def datetime_handler(x):
    if isinstance(x, datetime.datetime):
        return x.isoformat()
    return x

In [47]:
def datetime_handler(x):
    if isinstance(x, datetime.datetime):
        return x.isoformat()
    return x


start = datetime.datetime(2019,6,1)
end = datetime.datetime(2019,6,15)
#flights = fs.FlightService().get_flights(start,end)
# flights["ArrivalTimeUTC"] = flights["ArrivalTimeUTC"]
# flights["DepartTimeUTC"] = flights["DepartTimeUTC"]

paths = reco.RecommendationService().get_recommendations(flights,["JFK"],["LHR"],[],2,10)
paths

list_of_paths = []
for path in paths:
    list_of_flights = []
    for flights in path:
        list_of_flights.append(flights.to_dict())
    list_of_paths.append(list_of_flights)
list_of_paths = json.dumps(list_of_paths, default=datetime_handler)
list_of_paths


KeyError: 'current_path'

In [40]:
result = reco.RecommendationService().get_recommendations(flights, sources, targets, exclusions,2,10)
#result.to_json(orient='records')

result

KeyError: 'FlyTo'

In [25]:
q = reco.RecommendationService().get_initial_queue(sources)
items = pd.DataFrame.from_records([f.to_dict() for f in list(q.queue)])

assert(len(items) == 2)
items

,current_loc,current_path,price,state
0,JFK,[],0,FlightState.at_source
1,EWR,[],0,FlightState.at_source


In [14]:

current = q.get()
if current.state == reco.FlightState.at_completion:
    df = df.append(current.to_dict(), ignore_index=True)
else:
    reco.RecommendationService().process(q, current, flights, exclusions, targets, sources, 2, 10)

items = pd.DataFrame.from_records([f.to_dict() for f in list(q.queue)])

items


""


In [15]:
#df = df.append(current.to_dict(),ignore_index=True)
df



,current_loc,current_path,state
0,EWR,"[[DU, 2019-05-10 11:00:00, 2019-05-10 06:00:00...",FlightState.at_completion
1,EWR,"[[DU, 2019-05-05 11:00:00, 2019-05-05 06:00:00...",FlightState.at_completion


In [33]:
#at target
fs = flights
prev_arrival_time = current.current_path[len(current.current_path)-1]["ArrivalTimeUTC"]
int_destinations = fs[fs["FlyFrom"].isin(targets)]["FlyTo"].unique()
f1 = fs[(fs["FlyFrom"] == current.current_loc) & 
        (fs["DepartTimeUTC"] > prev_arrival_time + datetime.timedelta(days=2)) & 
        (fs["DepartTimeUTC"] < prev_arrival_time + datetime.timedelta(days=8)) &
        (~fs["FlyTo"].isin(exclusions)) & 
        ((fs["FlyTo"].isin(int_destinations)) | (fs["FlyTo"].isin(sources)))]
f1 = f1.assign(rn = f1.sort_values(['Price']).groupby(['FlyTo']).cumcount()+1).query('rn < 2')




,Airline,ArrivalTimeUTC,DepartTimeUTC,Duration,FlightNum,FlyFrom,FlyTo,Price,rn
3,DU,2019-05-18 11:00:00,2019-05-18 06:00:00,,123,LHR,EWR,450,1
4,DU,2019-05-18 11:00:00,2019-05-18 06:00:00,,123,LHR,ATL,200,1


In [73]:
current.current_path[]


[Airline                            DU
 ArrivalTimeUTC    2019-05-05 11:00:00
 DepartTimeUTC     2019-05-05 06:00:00
 Duration                             
 FlightNum                         456
 FlyFrom                           JFK
 FlyTo                             MIA
 Price                              50
 rn                                  1
 Name: 1, dtype: object]

In [19]:
#at int1

prev_arrival_time = current.current_path[len(current.current_path)-1]["ArrivalTimeUTC"]
fs = flights
f1 = fs[(fs["FlyFrom"] == current.current_loc) & (fs["FlyTo"].isin(targets)) & 
        (fs["DepartTimeUTC"] > prev_arrival_time + datetime.timedelta(days=2)) & 
        (fs["DepartTimeUTC"] < prev_arrival_time + datetime.timedelta(days=3))]
f1 = f1.assign(rn = f1.sort_values(['Price']).groupby(['FlyTo']).cumcount()+1).query('rn < 2')

for index, first_stop in f1.iterrows():
    new_path = current.current_path.copy()
    new_path.append(first_stop)

    q.put(reco.FlightItem(first_stop["FlyTo"], new_path, reco.FlightState.at_target))
    
for elem in list(q.queue):
    print(str(elem))

AttributeError: 'FlightItem' object has no attribute 'print'